In [6]:
from ngsolve import *
from mpi4py import MPI
from netgen.meshing import Mesh
from netgen.occ import unit_square
import meshio
from ngsolve import *
from ngsolve.webgui import Draw 
from netgen.meshing import Mesh as NGMesh
import netgen
from netgen.geom2d import SplineGeometry

In [ ]:
# geo = SplineGeometry()

# geo.AddRectangle(p1=(0,0),
#                  p2=(2,1),
#                  bc="rectangle",
#                  leftdomain=1,
#                  rightdomain=0)

# msh = Mesh(geo.GenerateMesh(maxh=0.1))

msh = Mesh(unit_square.GenerateMesh(maxh=0.05))

msh.ngmesh.Save("meshes/square/mesh.vol")


Draw(msh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [ ]:
fes = H1(msh, order=1, dirichlet="left|right")

# Trial and test functions
u = fes.TrialFunction()
v = fes.TestFunction()

# Right-hand side
g = sin(6*x)

In [9]:
a = BilinearForm(fes, symmetric=True)
a += grad(u)*grad(v)*dx

f = LinearForm(fes)
f += 10 * g * v * ds("top")
f += 10 * g * v * ds("bottom")

# f += 32 * (y*(1-y)+x*(1-x)) * v * dx

pre = preconditioners.Local(a)
a.Assemble()
f.Assemble()

gfu = GridFunction(fes)

In [10]:
inv = CGSolver(a.mat, pre.mat)
gfu.vec.data = inv*f.vec

Draw(gfu)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [ ]:
vtk = VTKOutput(msh,coefs=[gfu],names=["sol"],filename="output/square/poisson_square_ngsolve",subdivision=0)
vtk.Do()

'output/square/poisson_square_ngsolve'